<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Select-events" data-toc-modified-id="Select-events-0.1">Select events</a></span><ul class="toc-item"><li><span><a href="#Save-selected-events-to-file" data-toc-modified-id="Save-selected-events-to-file-0.1.1">Save selected events to file</a></span></li></ul></li><li><span><a href="#Align-clusters" data-toc-modified-id="Align-clusters-0.2">Align clusters</a></span></li><li><span><a href="#Add-tracks-and-vertex-to-events" data-toc-modified-id="Add-tracks-and-vertex-to-events-0.3">Add tracks and vertex to events</a></span><ul class="toc-item"><li><span><a href="#Save-tracked-events-to-file" data-toc-modified-id="Save-tracked-events-to-file-0.3.1">Save tracked events to file</a></span></li><li><span><a href="#Opening-angle" data-toc-modified-id="Opening-angle-0.3.2">Opening angle</a></span></li><li><span><a href="#DCA-between-two-tracks" data-toc-modified-id="DCA-between-two-tracks-0.3.3">DCA between two tracks</a></span></li><li><span><a href="#DCA-between-vertex-and-[0,0,0]" data-toc-modified-id="DCA-between-vertex-and-[0,0,0]-0.3.4">DCA between vertex and [0,0,0]</a></span></li></ul></li></ul></li><li><span><a href="#Very-clean-events" data-toc-modified-id="Very-clean-events-1">Very clean events</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Vertex-positions" data-toc-modified-id="Vertex-positions-1.0.1">Vertex positions</a></span></li><li><span><a href="#Position-shift-between-layers" data-toc-modified-id="Position-shift-between-layers-1.0.2">Position shift between layers</a></span></li><li><span><a href="#Correlation-between-ALPIDEs-after-all-selections" data-toc-modified-id="Correlation-between-ALPIDEs-after-all-selections-1.0.3">Correlation between ALPIDEs after all selections</a></span></li></ul></li></ul></li></ul></div>

In [ ]:
import math

import ROOT

from uits3_krakow22.src.Event import Event
from uits3_krakow22.src.Cluster import Cluster
from uits3_krakow22.src.Track import Track
from uits3_krakow22.src.Vertex import Vertex
from uits3_krakow22.src.Utils import *

from ipywidgets import IntProgress
import matplotlib.pyplot as plt
from IPython.display import display,HTML,clear_output
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))

### Select events

In [ ]:
path = "/home/berki/Software/uits3_krakow22/run456195948_221112200957.pkl"
nEvents = -1
selectedEvents = []
if nEvents > 0 :
    f = IntProgress(min=0, max=nEvents, description="Processing:")
    display(f)

for i,event in enumerate(readEvents(path, nEvents=nEvents)):
    if nEvents > 0 and i>0 and i%1e4 == 0: f.value += 1e4
    eventStatus = True
    for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]:
        clusters = event.selectDetector(alpide)
        selectedClusters = [cluster for cluster in clusters if 15 > cluster.size > 7]
        if len(selectedClusters) != 1:
            eventStatus = False
            break
    if eventStatus : selectedEvents.append(event)
        
if f : f.bar_style = "success"
print("Found events:",len(selectedEvents),", Fraction:",len(selectedEvents)*100/nEvents,"%")

#### Save selected events to file

In [ ]:
path_selected = "/home/berki/Software/uits3_krakow22/selectedEvents.pkl"
save2pickle(selectedEvents,path_selected)

### Align clusters

In [ ]:
displacements = {
        "ALPIDE_0" : [0,0,0],
        "ALPIDE_1" : [1.0, 0, 0],
        "ALPIDE_2" : [0,0,0],
        "ALPIDE_3" : [0.1875, 0.1875,0],
        "ALPIDE_4" : [0.0, -0.03125, 0] 
}

displacements0 = {
    "ALPIDE_0" : [0,0,0], # anchor
    "ALPIDE_1" : [1.156+0.05531,0.2837-0.03655,0],
    "ALPIDE_2" : [0.06475+0.2488,0.08900-0.03316,0],
    "ALPIDE_3" : [1.426-0.1012,0.7004-1.116,0],
    "ALPIDE_4" : [0.5261+0.5159,1.102-1.372,0],
}

displacements1 = {
    "ALPIDE_0" : [-1.211,-0.2472,0], 
    "ALPIDE_1" : [0,0,0],# anchor
    "ALPIDE_2" : [-0.7214,0.4154,0],
    "ALPIDE_3" : [0.4301,-0.02330,0],
    "ALPIDE_4" : [0.48375,0.01058,0],
}

displacements2 = {
    "ALPIDE_0" : [-0.3135,-0.05585,0], 
    "ALPIDE_1" : [0.7214,-0.4154,0],
    "ALPIDE_2" : [0,0,0],# anchor
    "ALPIDE_3" : [0.8794,-0.1445,0],
    "ALPIDE_4" : [0.9993,-0.1880,0],
}

displacements3 = {
    "ALPIDE_0" : [-1.321,0.4156,0], 
    "ALPIDE_1" : [-0.4301,0.02330,0],
    "ALPIDE_2" : [-0.8794,0.1445,0],
    "ALPIDE_3" : [0,0,0],# anchor
    "ALPIDE_4" : [-0.1492,0.8093,0],
}

displacements4 = {
    "ALPIDE_0" : [-1.042,0.2698,0], 
    "ALPIDE_1" : [-0.4837,-0.01058,0],
    "ALPIDE_2" : [-0.9960,0.1885,0],
    "ALPIDE_3" : [0.1492,-0.80930,0],
    "ALPIDE_4" : [0,0,0],# anchor
}



In [ ]:
events= []
for event in loadEvents(path_selected):
    for cluster in event.clusters:
        cluster.alignLocal(displacements4.get(cluster.detector))
    events.append(event)

### Add tracks and vertex to events

In [ ]:
path_selected = "/home/berki/Software/uits3_krakow22/selectedEvents.pkl"
events= []
for event in loadEvents(path_selected):
    TrackLeft = Track()
    TrackLeft.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_3","ALPIDE_4"]])
    TrackRight = Track()
    TrackRight.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
    vertex = Vertex()
    vertex.fromTracks([TrackLeft,TrackRight])
    event.addTrack(TrackLeft)
    event.addTrack(TrackRight)
    event.vertex = vertex
    events.append(event)

#### Save tracked events to file

In [ ]:
path_tracked = "/home/berki/Software/uits3_krakow22/trackedEvents.pkl"
save2pickle(events,path_tracked)

#### Opening angle

In [ ]:
%jsroot on
hOpeningAngle = ROOT.TH1F("Angle","Angle",300,80,100)
hOpeningAngle.GetXaxis().SetTitle("Opening angle between two tracks [°]")
hOpeningAngle.GetYaxis().SetTitle("Count")
for event in events:
    hOpeningAngle.Fill(event.vertex.openingAngle*360/(2*math.pi))
canvas = ROOT.TCanvas()
canvas.SetGridx(2)
hOpeningAngle.Draw("")
canvas.Draw()

In [ ]:
%jsroot on
hRMS = ROOT.TH1F("RMS cluster","RMS Cluster",22*5,-0.1,2.1)
hRMS.GetXaxis().SetTitle("RMS(cluster distance in track) [mm]")
hRMS.GetYaxis().SetTitle("Count")
for event in events:
    for track in event.tracks:
        if track.nClusters == 3:
            hRMS.Fill(track.rms)
canvas = ROOT.TCanvas()
canvas.SetGridx(2)
hRMS.Draw("")
canvas.Draw()
print("Mean",np.mean(rmsVals),"Med",np.median(rmsVals))

#### DCA between two tracks

In [ ]:
%jsroot on
hDCA = ROOT.TH1F("DCA","DCA between tracks",500,-0.1,5.1)
hDCA.GetXaxis().SetTitle("DCA between the tracks from 2 arms [mm]")
hDCA.GetYaxis().SetTitle("Count")
for event in events:
    #if abs(event.vertex.openingAngle - math.pi/2) < 0.05:
    if 88 < event.vertex.openingAngle*360/(2*math.pi) < 92:
        hDCA.Fill(event.vertex.dca)
canvas = ROOT.TCanvas()
canvas.SetGridx(2)
hDCA.Draw()
canvas.Draw()

#### DCA between vertex and [0,0,0]
Should probably use [0,0,-10.2] as origin as 0 along beam axis is currently the middle of the first layer

In [ ]:
%jsroot on
hDCA2origin = ROOT.TH1F("DCAtoOrigin","DCA to (0,0,0)",400,12,26)
hDCA2origin.GetXaxis().SetTitle("Distance of vertex to (0,0,0) [mm]")
hDCA2origin.GetYaxis().SetTitle("Count")
for event in events:
    if 86 < event.vertex.openingAngle*360/(2*math.pi) < 90:
        hDCA2origin.Fill(event.vertex.dca2origin)
canvas = ROOT.TCanvas()
canvas.SetGridx(2)
hDCA2origin.Draw()
canvas.Draw()

## Very clean events

In [ ]:
verySelectedEvents = []
for event in loadEvents(path_tracked):
    if 89 < event.vertex.openingAngle*360/(2*math.pi) < 92 and abs(event.vertex.dca2origin - 18.75) < 2 and event.vertex.dca < 2:
        verySelectedEvents.append(event)


In [ ]:
len(verySelectedEvents)
path_clean = "/home/berki/Software/uits3_krakow22/cleanEvents.pkl"
save2pickle(verySelectedEvents,path_clean)

#### Vertex positions

In [ ]:
%jsroot on
hVertexXY = ROOT.TH2F("VertexXY","VertexXY",100,-5,5,100,-5,5)
hVertexXY.GetXaxis().SetTitle("Vertex X position [mm]")
hVertexXY.GetYaxis().SetTitle("Vertex Y position [mm]")
hVertexXZ = ROOT.TH2F("VertexXZ","VertexXZ",100,-5,5,100,-23,-15)
hVertexXZ.GetXaxis().SetTitle("Vertex X position [mm]")
hVertexXZ.GetYaxis().SetTitle("Vertex Z position [mm]")
hVertexYZ = ROOT.TH2F("VertexYZ","VertexYZ",100,-5,5,100,-23,-15)
hVertexYZ.GetXaxis().SetTitle("Vertex Y position [mm]")
hVertexYZ.GetYaxis().SetTitle("Vertex Z position [mm]")
for event in events:
    if 86 < (event.vertex.openingAngle*360/(2*math.pi)) < 90 and event.vertex.dca < 3 and 16 < event.vertex.dca2origin < 20:
        hVertexXY.Fill(-event.vertex.point[0],event.vertex.point[1])
        hVertexXZ.Fill(-event.vertex.point[0],event.vertex.point[2])
        hVertexYZ.Fill(event.vertex.point[1],event.vertex.point[2])
    
canvas = ROOT.TCanvas("","",1200,400)
canvas.Divide(3)
canvas.cd(1)
ROOT.gPad.SetGrid(2)
hVertexXY.Draw("COLZ")
canvas.cd(2)
ROOT.gPad.SetGrid(2)
hVertexXZ.Draw("COLZ")
canvas.cd(3)
ROOT.gPad.SetGrid(2)
hVertexYZ.Draw("COLZ")
canvas.Draw()

#### Position shift between layers

In [ ]:
%jsroot on
alpide1 = "ALPIDE_1"
alpide2 = "ALPIDE_2"
hShiftXY = ROOT.TH2F("ShiftXY","ShiftXY",150,-15,15,75,-7.5,7.5)
hShiftX = ROOT.TH1F("ShiftX","ShiftX",150,-15,15)
hShiftY = ROOT.TH1F("ShiftY","ShiftY",75,-7.5,7.5)
for event in events:
    if abs(event.vertex.openingAngle - math.pi/2) < 0.05 and event.vertex.dca < 2:
        hShiftXY.Fill(event.selectDetector(alpide1)[0].localPos[0]-event.selectDetector(alpide2)[0].localPos[0],
                      event.selectDetector(alpide1)[0].localPos[1]-event.selectDetector(alpide2)[0].localPos[1])
        hShiftX.Fill(event.selectDetector(alpide1)[0].localPos[0]-event.selectDetector(alpide2)[0].localPos[0])
        hShiftY.Fill(event.selectDetector(alpide1)[0].localPos[1]-event.selectDetector(alpide2)[0].localPos[1])
canvas = ROOT.TCanvas("","",1200,400)
canvas.Divide(3)
canvas.cd(1)
hShiftXY.Draw("COLZ")
canvas.cd(2)
hShiftX.Draw()
canvas.cd(3)
hShiftY.Draw()
canvas.Draw()

#### Correlation between ALPIDEs after all selections

In [ ]:
%jsroot on

nBins = {"x-Axis" : 600, "y-Axis" : 300}
nRange = {"x-Axis" : 15, "y-Axis" : 7.5}
axis = {"x-Axis":0, "y-Axis":1}

detector1 = "ALPIDE_0"
detector2 = "ALPIDE_4"
            
hCorrXX = ROOT.TH2F("Corr_"+detector1+"_x-Axis_"+detector2+"_x-Axis","Corr_"+detector1+"_x-Axis_"+detector2+"_x-Axis", nBins.get("x-Axis"),-nRange.get("x-Axis"),nRange.get("x-Axis"),nBins.get("x-Axis"),-nRange.get("x-Axis"),nRange.get("x-Axis"))
hCorrYY = ROOT.TH2F("Corr_"+detector1+"_y-Axis_"+detector2+"_y-Axis","Corr_"+detector1+"_y-Axis_"+detector2+"_y-Axis", nBins.get("y-Axis"),-nRange.get("y-Axis"),nRange.get("y-Axis"),nBins.get("y-Axis"),-nRange.get("y-Axis"),nRange.get("y-Axis"))
hCorrXX.SetXTitle(detector1 + " - x-Axis [mm]")
hCorrXX.SetYTitle(detector2 + " - x-Axis [mm]")
hCorrYY.SetXTitle(detector1 + " - y-Axis [mm]")
hCorrYY.SetYTitle(detector2 + " - y-Axis [mm]")

for event in events:
    if abs(event.vertex.openingAngle - math.pi/2) < 0.05 and event.vertex.dca < 1:
        for cluster1 in event.selectDetector(detector1):
            for cluster2 in event.selectDetector(detector2):
                hCorrXX.Fill(cluster1.localPos[0],cluster2.localPos[0])
                hCorrYY.Fill(cluster1.localPos[1],cluster2.localPos[1])
                

canvas = ROOT.TCanvas("","",1000,500)
canvas.Divide(2)
canvas.cd(1)
hCorrXX.Draw("COLZ")
canvas.cd(2)
hCorrYY.Draw("COLZ")
canvas.Draw()

# Scan alignment

In [ ]:
selectedEvents = [event for event in readEvents("/home/berki/Software/uits3_krakow22/selectedEvents.pkl")]

In [ ]:
import numpy as np
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]

displacements = [{
    "ALPIDE_0" : [0,0,0], # anchor
    "ALPIDE_1" : [1,0,0],
    "ALPIDE_2" : [0,0,0],
}]

#displacements = [[i/10,j/10] for i in range(10,15,1) for j in range(0,5,1)]
#displacements = [[1.2113099999999999, 0.24715]]
performance = []
for displacement in displacements:
    rmsVals = []
    for event in readEvents("/home/berki/Software/uits3_krakow22/selectedEvents.pkl"):
        #event.selectDetector("ALPIDE_1")[0].alignLocal(displacement.get("A"))
        track = Track()
        track.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)) for alpide in detectors])
        rmsVals.append(track.rms)
    goodness = np.median(rmsVals)
    performance.append(goodness)
    print(displacement,":",goodness)

Going to process all events.
{'ALPIDE_0': [0, 0, 0], 'ALPIDE_1': [1, 0, 0], 'ALPIDE_2': [0, 0, 0]} : 0.034732187302410344

In [ ]:
import matplotlib.pyplot as plt
plt.hist(rmsVals,1000)


In [ ]:
myCorners = [[i,j] for i in [-1,1] for j in [-1,1]]
myCorners

In [ ]:
def evalDisplacement(displacement):
    dcaVals = []
    for event in loadEvents("/home/berki/Software/uits3_krakow22/cleanEvents.pkl"):
        TrackLeft = Track()
        TrackLeft.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)) for alpide in ["ALPIDE_3","ALPIDE_4"]])
        TrackRight = Track()
        TrackRight.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
        vertex = Vertex()
        vertex.fromTracks([TrackLeft,TrackRight])
        dcaVals.append(vertex.dca)
    goodness = np.median(dcaVals)
    print("Evaluated",displacement,". Got",goodness)
    return goodness

def findBest(x1,x2,y1,y2,step):
    print("Round",step)
    corners = [[x,y,0] for x in [x1,x2] for y in [y1,y2]]
    displacements = [{
        "ALPIDE_0" : [0,0,0],
        "ALPIDE_1" : [1.03125, -0.96875, 0],
        "ALPIDE_2" : [0,0,0],
        "ALPIDE_3" : [0.1875, 0.1875,0],
        "ALPIDE_4" : [0.0, -0.03125, 0],
    } for corner in corners]

    results = [evalDisplacement(displacement) for displacement in displacements]
    winner = results.index(min(results))
    print("Winner of round",step,"is",corners[winner],"with",results[winner])
    if step <= 0 :
        return [results[winner],corners[winner]]
    else : 
        findBest(corners[winner][0],(x1+x2)/2,corners[winner][1],(y1+y2)/2, step-1)
    

In [ ]:
findBest(-2,2,-2,2,6)

[0.375, 0.375, 0]

In [ ]:
def loadEvents(filename):
    with open(filename, "rb") as f:
        events = pickle.load(f)
        return events